In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd

gppd.set_interactive()
pd.set_option('display.max_colwidth',1000)

In [2]:
env = gp.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-03


## Data

In [3]:
import os

data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "input_data.xlsx")
waste_source_data = pd.read_excel(data_path, sheet_name='Waste Source')
shredder_data = pd.read_excel(data_path, sheet_name='E-waste Collector')
oxide_producer_data = pd.read_excel(data_path, sheet_name='Oxide Producer')
fluoride_producer_data = pd.read_excel(data_path, sheet_name='Flouride Producer')
metal_producer_data = pd.read_excel(data_path, sheet_name='Metal Producer')
magnet_producer_data = pd.read_excel(data_path, sheet_name='Magnet Producer')

# Create dictionaries for shredder and oxide producer coordinates
waste_source = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in waste_source_data.iterrows()}
shredders = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in shredder_data.iterrows()}
oxide_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in oxide_producer_data.iterrows()}
fluoride_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in fluoride_producer_data.iterrows()}
metal_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in metal_producer_data.iterrows()}
magnet_producers = {row['Code']: (row['Latitude'], row['Longitude']) for index, row in magnet_producer_data.iterrows()}

In [4]:

price = pd.read_excel(data_path, sheet_name='REE Price')
price.set_index("Year", inplace = True)
price = price["Price"]
price.index.name = "year"
price

year
Y1     122000.000000
Y2     126001.600000
Y3     130134.452480
Y4     134402.862521
Y5     138811.276412
Y6     143364.286278
Y7     148066.634868
Y8     152923.220492
Y9     157939.102124
Y10    163119.504674
Name: Price, dtype: float64

In [5]:
MARR = 0.08

In [6]:
waste_volume = pd.read_excel(data_path, sheet_name='Waste Volume')
waste_volume.set_index('Code', inplace=True)
waste_volume = waste_volume['volume (ton)']
waste_volume.index.name = "from"
waste_volume.head()

from
W1    8258.035
W2    3820.914
W3    2664.452
W4    2314.157
W5    1650.070
Name: volume (ton), dtype: float64

In [7]:
oc_shredder = pd.read_excel(data_path, sheet_name='Shredder_OC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
oc_shredder = pd.melt(oc_shredder, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="shredder_oc")
oc_shredder.set_index(["Code", "Year"], inplace=True)
oc_shredder = oc_shredder["shredder_oc"]
oc_shredder.index.names = ["from", "year"]
oc_shredder.head()

from  year
S01   Y1      1927.000000
S02   Y1      2098.880653
S03   Y1      2202.977387
S04   Y1      1554.188442
S05   Y1      2202.977387
Name: shredder_oc, dtype: float64

In [8]:
oc_oxide = pd.read_excel(data_path, sheet_name='Oxide_OC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
oc_oxide = pd.melt(oc_oxide, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="oxide_oc")
oc_oxide.set_index(["Code", "Year"], inplace=True)
oc_oxide = oc_oxide["oxide_oc"]
oc_oxide.index.names = ["from", "year"]
oc_oxide.head()

from  year
OP01  Y1      7789.251256
OP02  Y1      7702.428392
OP03  Y1      8806.319095
OP04  Y1      7702.428392
OP05  Y1      7255.910804
Name: oxide_oc, dtype: float64

In [9]:
oc_fluoride = pd.read_excel(data_path, sheet_name='Fluoride_OC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
oc_fluoride = pd.melt(oc_fluoride, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="fluoride_oc")
oc_fluoride.set_index(["Code", "Year"], inplace=True)
oc_fluoride = oc_fluoride['fluoride_oc']
oc_fluoride.index.names = ["from", "year"]
oc_fluoride.head()

from  year
FP01  Y1      5289.000000
FP02  Y1      5450.037051
FP03  Y1      5450.037051
FP04  Y1      5308.402054
FP05  Y1      5582.941123
Name: fluoride_oc, dtype: float64

In [10]:
oc_ree = pd.read_excel(data_path, sheet_name='REE_OC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
oc_ree = pd.melt(oc_ree, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="ree_oc")
oc_ree.set_index(["Code", "Year"], inplace=True)
oc_ree = oc_ree['ree_oc']
oc_ree.index.names = ["from", "year"]
oc_ree.head()

from  year
MP01  Y1      32868.000000
MP02  Y1      32496.376884
MP03  Y1      36460.356784
MP01  Y2      33946.070400
MP02  Y2      33562.258046
Name: ree_oc, dtype: float64

In [11]:
fc_shredder = pd.read_excel(data_path, sheet_name='Shredder_FC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
fc_shredder = pd.melt(fc_shredder, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="shredder_fc")
fc_shredder.set_index(["Code", "Year"], inplace=True)
fc_shredder = fc_shredder['shredder_fc']
fc_shredder.index.names = ["from", "year"]
fc_shredder.head()

from  year
S01   Y1      157225.000000
S02   Y1      162674.707126
S03   Y1      180466.956522
S04   Y1      150901.820652
S05   Y1      180466.956522
Name: shredder_fc, dtype: float64

In [12]:
fc_oxide = pd.read_excel(data_path, sheet_name='Oxide_FC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
fc_oxide = pd.melt(fc_oxide, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="oxide_fc")
fc_oxide.set_index(["Code", "Year"], inplace=True)
fc_oxide = fc_oxide['oxide_fc']
fc_oxide.index.names = ["from", "year"]
fc_oxide.head()

from  year
OP01  Y1      147726.376274
OP02  Y1      156627.617949
OP03  Y1      162533.249446
OP04  Y1      153931.568788
OP05  Y1      153204.063459
Name: oxide_fc, dtype: float64

In [13]:
fc_fluoride = pd.read_excel(data_path, sheet_name='Fluoride_FC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
fc_fluoride = pd.melt(fc_fluoride, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="fluoride_fc")
fc_fluoride.set_index(["Code", "Year"], inplace=True)
fc_fluoride = fc_fluoride['fluoride_fc']
fc_fluoride.index.names = ["from", "year"]
fc_fluoride.head()

from  year
FP01  Y1       89884.761189
FP02  Y1      109730.760797
FP03  Y1      106554.643138
FP04  Y1       80697.383070
FP05  Y1       99754.089025
Name: fluoride_fc, dtype: float64

In [14]:
fc_ree = pd.read_excel(data_path, sheet_name='REE_FC')
year_index = ["Y1", "Y2", "Y3", "Y4", "Y5", "Y6", "Y7", "Y8", "Y9", "Y10"]
fc_ree = pd.melt(fc_ree, id_vars=["Code"], value_vars=year_index, var_name="Year", value_name="ree_fc")
fc_ree.set_index(["Code", "Year"], inplace=True)
fc_ree= fc_ree['ree_fc']
fc_ree.index.names = ["from", "year"]
fc_ree.head()

from  year
MP01  Y1      683356.326646
MP02  Y1      794877.576072
MP03  Y1      665005.357301
MP01  Y2      705770.414160
MP02  Y2      820949.560567
Name: ree_fc, dtype: float64

In [15]:
conv1 = 0.089 # conversion factor from shredded HDD to initial oxalate
conv2 = 0.083 # conversion factor from oxalate to oxides
conv3 = 1.074 # conversion fcator from oxides to fluorides
conv4 = 0.72 # conversion factor from fluorides to metals
w = 0.3 # Tax credit as percentage of capital cost

In [16]:
inflation_rate = 0.0328

In [17]:
shredder_cap = pd.read_excel(data_path, sheet_name='Shredder Capacity')
oxide_cap = pd.read_excel(data_path, sheet_name='Oxide Capacity')
fluoride_cap = pd.read_excel(data_path, sheet_name='Fluoride Capacity')
metal_cap = pd.read_excel(data_path, sheet_name='REE Capacity')

In [18]:
shredder_cap.set_index("Code", inplace=True)
shredder_cap = shredder_cap['Capacity']
shredder_cap.index.name = "from"
shredder_cap

from
S01      648.000000
S02       21.277877
S03     4610.206595
S04     7979.203721
S05     8865.781913
S06     3989.601861
S07     1595.840744
S08     1844.082638
S09    23051.032973
Name: Capacity, dtype: float64

In [19]:
'''
shredder_cap_copy = pd.read_excel(data_path, sheet_name='Shredder Capacity')
shredder_cap_copy.set_index("Code", inplace=True)
shredder_cap_copy = shredder_cap_copy['Capacity']
shredder_cap_copy.index.name = "to"
shredder_cap_copy
'''

'\nshredder_cap_copy = pd.read_excel(data_path, sheet_name=\'Shredder Capacity\')\nshredder_cap_copy.set_index("Code", inplace=True)\nshredder_cap_copy = shredder_cap_copy[\'Capacity\']\nshredder_cap_copy.index.name = "to"\nshredder_cap_copy\n'

In [20]:
oxide_cap.set_index("Code", inplace=True)
oxide_cap = oxide_cap['Capacity']
oxide_cap.index.name = "from"
oxide_cap

from
OP01    54
OP02    54
OP03    54
OP04    54
OP05    54
        ..
OP61    54
OP62    54
OP63    54
OP64    54
OP65    54
Name: Capacity, Length: 65, dtype: int64

In [21]:
fluoride_cap.set_index("Code", inplace=True)
fluoride_cap = fluoride_cap['Capacity']
fluoride_cap.index.name = "from"
fluoride_cap

from
FP01    58
FP02    58
FP03    58
FP04    58
FP05    58
FP06    58
Name: Capacity, dtype: int64

In [22]:
metal_cap.set_index("Code", inplace=True)
metal_cap = metal_cap['Capacity']
metal_cap.index.name = "from"
metal_cap

from
MP01      41
MP02     150
MP03    1000
Name: Capacity, dtype: int64

In [23]:
shredder_to_oxide_tc = pd.read_excel(data_path, sheet_name='Shredder-Oxide TC')
shredder_to_oxide_tc.head()

,index,OP01,OP02,OP03,OP04,OP05,OP06,OP07,OP08,OP09,...,OP56,OP57,OP58,OP59,OP60,OP61,OP62,OP63,OP64,OP65
0,S01,391.863447,549.665602,656.668150,555.767979,104.365417,219.604161,321.589695,308.788572,316.499981,...,414.914909,1.321138,20.458399,63.705196,640.175848,92.734438,825.533995,310.679011,204.524265,182.347670
1,S02,394.383810,571.502181,678.504505,577.604558,86.060969,240.872333,324.109834,324.303329,328.432724,...,436.183081,24.171499,20.873799,85.541775,650.879392,115.978277,847.370351,311.623672,188.199734,204.184026
2,S03,386.280935,548.512682,655.515230,554.615283,107.000311,214.021649,316.006959,303.206060,310.917246,...,409.332173,5.340251,14.934047,62.552500,627.890116,96.945928,824.381075,316.947817,210.793070,181.194751
3,S04,391.572645,732.009917,839.012465,738.112518,146.208980,308.572259,338.743460,342.752507,343.063442,...,457.053972,235.765618,242.562566,298.027539,835.244560,324.689873,935.564244,392.485317,103.023028,397.487566
4,S05,300.029379,477.591336,584.593883,483.693713,112.249295,134.595225,230.000572,223.779413,231.490822,...,329.905750,85.505760,79.675619,56.437372,580.825978,158.262070,753.459729,397.113102,277.202058,142.588042


In [24]:
column_names = shredder_to_oxide_tc.columns.delete(0)
shredder_to_oxide_tc = pd.melt(shredder_to_oxide_tc, id_vars=["index"], value_vars=column_names, var_name="to", value_name="Y1").rename({'index':'from'}, axis = 1)

for i in range(2, 11):
    var_name = "Y" + str(i)
    shredder_to_oxide_tc[var_name] = shredder_to_oxide_tc.iloc[:, i] * (1 + inflation_rate)

shredder_to_oxide_tc.head()

,from,to,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,S01,OP01,391.863447,404.716568,417.991272,431.701385,445.861191,460.485438,475.589360,491.188691,507.299680,523.939110
1,S02,OP01,394.383810,407.319598,420.679681,434.477975,448.728852,463.447159,478.648226,494.347887,510.562498,527.308948
2,S03,OP01,386.280935,398.950950,412.036541,425.551339,439.509423,453.925332,468.814083,484.191185,500.072656,516.475039
3,S04,OP01,391.572645,404.416227,417.681080,431.381019,445.530317,460.143711,475.236425,490.824179,506.923212,523.550294
4,S05,OP01,300.029379,309.870342,320.034089,330.531207,341.372631,352.569653,364.133938,376.077531,388.412874,401.152816


In [25]:
shredder_to_oxide_tc = pd.melt(shredder_to_oxide_tc, id_vars=["from", "to"], value_vars=year_index, var_name="year", value_name="cost")
shredder_to_oxide_tc.set_index(["from", "to", "year"], inplace=True)

In [26]:
shredder_to_oxide_tc = shredder_to_oxide_tc["cost"]
shredder_to_oxide_tc.head()

from  to    year
S01   OP01  Y1      391.863447
S02   OP01  Y1      394.383810
S03   OP01  Y1      386.280935
S04   OP01  Y1      391.572645
S05   OP01  Y1      300.029379
Name: cost, dtype: float64

In [27]:
oxide_to_fluoride_tc = pd.read_excel(data_path, sheet_name='Oxide-Fluoride TC')

column_names = oxide_to_fluoride_tc.columns.delete(0)
oxide_to_fluoride_tc = pd.melt(oxide_to_fluoride_tc, id_vars=["index"], value_vars=column_names, var_name="to", value_name="Y1").rename({'index':'from'}, axis = 1)

for i in range(2, 11):
    var_name = "Y" + str(i)
    oxide_to_fluoride_tc[var_name] = oxide_to_fluoride_tc.iloc[:, i] * (1 + inflation_rate)

oxide_to_fluoride_tc.head()

oxide_to_fluoride_tc = pd.melt(oxide_to_fluoride_tc, id_vars=["from", "to"], value_vars=year_index, var_name="year", value_name="cost")
oxide_to_fluoride_tc.set_index(["from", "to", "year"], inplace=True)

In [28]:
oxide_to_fluoride_tc = oxide_to_fluoride_tc['cost']
oxide_to_fluoride_tc.head()

from  to    year
OP01  FP01  Y1      498.508979
OP02  FP01  Y1      624.991040
OP03  FP01  Y1      731.486025
OP04  FP01  Y1      630.205799
OP05  FP01  Y1      190.370462
Name: cost, dtype: float64

In [29]:
waste_to_shredder_tc = pd.read_excel(data_path, sheet_name='Waste-Shredder TC')

column_names = waste_to_shredder_tc.columns.delete(0)
waste_to_shredder_tc = pd.melt(waste_to_shredder_tc, id_vars=["index"], value_vars=column_names, var_name="to", value_name="Y1").rename({'index':'from'}, axis = 1)

for i in range(2, 11):
    var_name = "Y" + str(i)
    waste_to_shredder_tc[var_name] = waste_to_shredder_tc.iloc[:, i] * (1 + inflation_rate)

waste_to_shredder_tc.head()

waste_to_shredder_tc = pd.melt(waste_to_shredder_tc, id_vars=["from", "to"], value_vars=year_index, var_name="year", value_name="cost")
waste_to_shredder_tc.set_index(["from", "to", "year"], inplace=True)

In [30]:
waste_to_shredder_tc = waste_to_shredder_tc['cost']
waste_to_shredder_tc.head()

from  to   year
W1    S01  Y1      254.297563
W2    S01  Y1      745.160898
W3    S01  Y1       67.257683
W4    S01  Y1      366.273056
W5    S01  Y1      612.445599
Name: cost, dtype: float64

In [31]:
fluoride_to_metal_tc = pd.read_excel(data_path, sheet_name='Fluoride-Metal TC')
fluoride_to_metal_tc.head()

column_names = fluoride_to_metal_tc.columns.delete(0)
fluoride_to_metal_tc = pd.melt(fluoride_to_metal_tc, id_vars=["index"], value_vars=column_names, var_name="to", value_name="Y1").rename({'index':'from'}, axis = 1)

for i in range(2, 11):
    var_name = "Y" + str(i)
    fluoride_to_metal_tc[var_name] = fluoride_to_metal_tc.iloc[:, i] * (1 + inflation_rate)

fluoride_to_metal_tc.head()

fluoride_to_metal_tc = pd.melt(fluoride_to_metal_tc, id_vars=["from", "to"], value_vars=year_index, var_name="year", value_name="cost")
fluoride_to_metal_tc.set_index(["from", "to", "year"], inplace=True)


In [32]:
fluoride_to_metal_tc = fluoride_to_metal_tc['cost']
fluoride_to_metal_tc.head()

from  to    year
FP01  MP01  Y1        1.466763
FP02  MP01  Y1      847.875228
FP03  MP01  Y1      847.480856
FP04  MP01  Y1      346.430934
FP05  MP01  Y1      222.823791
Name: cost, dtype: float64

In [33]:
metal_to_magnet_tc = pd.read_excel(data_path, sheet_name='Metal-Magnet TC')
metal_to_magnet_tc.head()

column_names = metal_to_magnet_tc.columns.delete(0)
metal_to_magnet_tc = pd.melt(metal_to_magnet_tc, id_vars=["index"], value_vars=column_names, var_name="to", value_name="Y1").rename({'index':'from'}, axis = 1)

for i in range(2, 11):
    var_name = "Y" + str(i)
    metal_to_magnet_tc[var_name] = metal_to_magnet_tc.iloc[:, i] * (1 + inflation_rate)

metal_to_magnet_tc.head()

metal_to_magnet_tc = pd.melt(metal_to_magnet_tc, id_vars=["from", "to"], value_vars=year_index, var_name="year", value_name="cost")
metal_to_magnet_tc.set_index(["from", "to", "year"], inplace=True)

In [34]:
metal_to_magnet_tc = metal_to_magnet_tc['cost']
metal_to_magnet_tc.head()

from  to     year
MP01  MAG01  Y1      370.090621
MP02  MAG01  Y1      426.428449
MP03  MAG01  Y1      232.849988
MP01  MAG02  Y1      512.366163
MP02  MAG02  Y1      473.535089
Name: cost, dtype: float64

In [35]:
is_shredder_incentive_eligible = shredder_data[["Code", "is_energy_community"]]
is_oxide_producer_incentive_eligible = oxide_producer_data[["Code", "is_energy_community"]]

is_shredder_incentive_eligible.set_index("Code", inplace = True)
is_oxide_producer_incentive_eligible.set_index("Code", inplace = True)

is_shredder_incentive_eligible.index.name = "from"
is_oxide_producer_incentive_eligible.index.name = "from"

is_shredder_incentive_eligible = is_shredder_incentive_eligible["is_energy_community"]
is_oxide_producer_incentive_eligible = is_oxide_producer_incentive_eligible["is_energy_community"]

In [36]:
is_shredder_incentive_eligible[:5]

from
S01    1
S02    0
S03    1
S04    1
S05    1
Name: is_energy_community, dtype: int64

## Model with no fixed charge

In [37]:
model_no_fixed_charge = gp.Model("Ewaste_no_fixed_charge", env=env)
model_no_fixed_charge.ModelSense = GRB.MAXIMIZE

## Decision Variables

In [38]:

flow_shredder_to_oxide = gppd.add_vars(
    model_no_fixed_charge,
    shredder_to_oxide_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_shredder_to_oxide"
)

flow_oxide_to_fluoride = gppd.add_vars(
    model_no_fixed_charge,
    oxide_to_fluoride_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_oxide_to_fluoride"
)

flow_fluoride_to_metal = gppd.add_vars(
    model_no_fixed_charge,
    fluoride_to_metal_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_fluoride_to_metal"
)

shredder_loc = list(shredders.keys())
waste_to_shredder_index = pd.MultiIndex.from_product([waste_volume.index, shredder_loc, year_index], names = ["from", "to", "year"])
flow_waste_to_shredder = gppd.add_vars(
    model_no_fixed_charge,
    waste_to_shredder_index,
    vtype = GRB.CONTINUOUS,
    name = "flow_waste_to_shredder"
)

flow_metal_to_magnet = gppd.add_vars(
    model_no_fixed_charge,
    metal_to_magnet_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_metal_to_magnet"
)

## Constraints

### Flow balance constraints

In [39]:
shredder_balance = gppd.add_constrs(
    model_no_fixed_charge,
    flow_shredder_to_oxide.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv1 * (flow_waste_to_shredder.groupby(['to', 'year']).sum()),
    name = "shredder_balance"
)

In [40]:
oxide_balance = gppd.add_constrs(
    model_no_fixed_charge,
    flow_oxide_to_fluoride.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv2 * (flow_shredder_to_oxide.groupby(['to', 'year']).sum()),
    name = "oxide_balance"
)

In [41]:
fluoride_balance = gppd.add_constrs(
    model_no_fixed_charge,
    flow_fluoride_to_metal.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv3 * (flow_oxide_to_fluoride.groupby(['to', 'year']).sum()),
    name = "fluoride_balance"
)

In [42]:
metal_balance = gppd.add_constrs(
    model_no_fixed_charge,
    flow_metal_to_magnet.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv4 * (flow_fluoride_to_metal.groupby(['to', 'year']).sum()),
    name = "metal_balance"
)

### Capacity constraint

In [43]:
waste_supply_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_waste_to_shredder.groupby(['from', 'year']).sum() - waste_volume,
    GRB.LESS_EQUAL,
    0,
    name = "waste_supply_limit"
)

In [44]:
shredder_supply_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_shredder_to_oxide.groupby(['from', 'year']).sum() - shredder_cap,
    GRB.LESS_EQUAL,
    0,
    name = "shredder_supply_limit"
)

In [45]:
oxide_supply_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_oxide_to_fluoride.groupby(['from', 'year']).sum() - oxide_cap,
    GRB.LESS_EQUAL,
    0,
    name = "oxide_supply_limit"
)

In [46]:
fluoride_supply_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_fluoride_to_metal.groupby(['from', 'year']).sum() - fluoride_cap,
    GRB.LESS_EQUAL,
    0,
    name = "fluoride_supply_limit"
)

In [47]:
metal_supply_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_metal_to_magnet.groupby(['from', 'year']).sum() - metal_cap,
    GRB.LESS_EQUAL,
    0,
    name = "metal_supply_limit"
)

In [48]:
'''
shredder_inbound_limit = gppd.add_constrs(
    model_no_fixed_charge,
    flow_waste_to_shredder.groupby(['to', 'year']).sum() - (shredder_cap_copy / conv1),
    GRB.LESS_EQUAL,
    0,
    name = "shredder_inbound_limit"
)
'''

'\nshredder_inbound_limit = gppd.add_constrs(\n    model_no_fixed_charge,\n    flow_waste_to_shredder.groupby([\'to\', \'year\']).sum() - (shredder_cap_copy / conv1),\n    GRB.LESS_EQUAL,\n    0,\n    name = "shredder_inbound_limit"\n)\n'

## Objective function

In [49]:
revenue = []
for t in year_index:
    revenue.append(price[t] * (flow_metal_to_magnet.loc[slice(None), slice(None), t]).sum())
revenue = pd.Series(revenue, index=year_index, name='revenue')

conversion_cost_shredder = []
for t in year_index:
    conversion_cost_shredder.append((oc_shredder.loc[slice(None), t] * flow_shredder_to_oxide.loc[slice(None), slice(None), t]).sum())
conversion_cost_shredder = pd.Series(conversion_cost_shredder, index=year_index, name='conversion_cost_shredder')

conversion_cost_oxide = []
for t in year_index:
    conversion_cost_oxide.append((oc_oxide.loc[slice(None), t] * flow_oxide_to_fluoride.loc[slice(None), slice(None), t]).sum())
conversion_cost_oxide = pd.Series(conversion_cost_oxide, index=year_index, name='conversion_cost_oxide')

conversion_cost_fluoride = []
for t in year_index:
    conversion_cost_fluoride.append((oc_fluoride.loc[slice(None), t] * flow_fluoride_to_metal.loc[slice(None), slice(None), t]).sum())
conversion_cost_fluoride = pd.Series(conversion_cost_fluoride, index=year_index, name='conversion_cost_fluoride')

conversion_cost_metal = []
for t in year_index:
    conversion_cost_metal.append((oc_ree.loc[slice(None), t] * flow_metal_to_magnet.loc[slice(None), slice(None), t]).sum())
conversion_cost_metal = pd.Series(conversion_cost_metal, index=year_index, name='conversion_cost_metal')

transportation_cost_shredder = []
for t in year_index:
    transportation_cost_shredder.append((shredder_to_oxide_tc.loc[slice(None), slice(None), t] * flow_shredder_to_oxide.loc[slice(None), slice(None), t]).sum())
transportation_cost_shredder = pd.Series(transportation_cost_shredder, index=year_index, name='transportation_cost_shredder')

transportation_cost_oxide = []
for t in year_index:
    transportation_cost_oxide.append((oxide_to_fluoride_tc.loc[slice(None), slice(None), t] * flow_oxide_to_fluoride.loc[slice(None), slice(None), t]).sum())
transportation_cost_oxide = pd.Series(transportation_cost_oxide, index=year_index, name='transportation_cost_oxide')

transportation_cost_fluoride = []
for t in year_index:
    transportation_cost_fluoride.append((fluoride_to_metal_tc.loc[slice(None), slice(None), t] * flow_fluoride_to_metal.loc[slice(None), slice(None), t]).sum())
transportation_cost_fluoride = pd.Series(transportation_cost_fluoride, index=year_index, name='transportation_cost_fluoride')

transportation_cost_metal = []
for t in year_index:
    transportation_cost_metal.append((metal_to_magnet_tc.loc[slice(None), slice(None), t] * flow_metal_to_magnet.loc[slice(None), slice(None), t]).sum())
transportation_cost_metal = pd.Series(transportation_cost_metal, index=year_index, name='transportation_cost_metal')

transportation_cost_waste_supplier = []
for t in year_index:
    transportation_cost_waste_supplier.append((waste_to_shredder_tc.loc[slice(None), slice(None), t] * flow_waste_to_shredder.loc[slice(None), slice(None), t]).sum())
transportation_cost_waste_supplier = pd.Series(transportation_cost_waste_supplier, index=year_index, name='transportation_cost_waste_supplier')

In [50]:
total_revenue = 0
for j, t in enumerate(year_index, start=1):
    total_revenue += (1 / (1+MARR)**j) * revenue[t]
    
total_conversion_cost_shredder = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_shredder += (1 / (1+MARR)**j) * conversion_cost_shredder[t]
    
total_conversion_cost_oxide = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_oxide += (1 / (1+MARR)**j) * conversion_cost_oxide[t]

total_conversion_cost_fluoride = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_fluoride += (1 / (1+MARR)**j) * conversion_cost_fluoride[t]

total_conversion_cost_metal = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_metal += (1 / (1+MARR)**j) * conversion_cost_metal[t]

total_transportation_cost_shredder = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_shredder += (1 / (1+MARR)**j) * transportation_cost_shredder[t]
    
total_transportation_cost_oxide = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_oxide += (1 / (1+MARR)**j) * transportation_cost_oxide[t]
    
total_transportation_cost_fluoride = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_fluoride += (1 / (1+MARR)**j) * transportation_cost_fluoride[t]
    
total_transportation_cost_metal = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_metal += (1 / (1+MARR)**j) * transportation_cost_metal[t]
    
total_transportation_cost_waste_supplier = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_waste_supplier += (1 / (1+MARR)**j) * transportation_cost_waste_supplier[t]

In [51]:
obj_exprs = total_revenue - total_conversion_cost_shredder - total_conversion_cost_oxide - total_conversion_cost_fluoride - total_conversion_cost_metal - \
            total_transportation_cost_shredder - total_transportation_cost_oxide - total_transportation_cost_fluoride - total_transportation_cost_metal - total_transportation_cost_waste_supplier
            
model_no_fixed_charge.setObjective(obj_exprs, sense=GRB.MAXIMIZE)
model_no_fixed_charge.update()

## Solve no fixed charge problem

In [52]:
model_no_fixed_charge.reset()
model_no_fixed_charge.optimize()

Discarded solution information
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 18750 rows, 163950 columns and 337830 nonzeros
Model fingerprint: 0x02652620
Coefficient statistics:
  Matrix range     [8e-02, 1e+00]
  Objective range  [5e-01, 8e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+04]
Presolve removed 30 rows and 180 columns
Presolve time: 0.30s
Presolved: 18720 rows, 163770 columns, 337440 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 AA' NZ     : 1.745e+05
 Factor NZ  : 2.132e+05 (roughly 80 MB of memory)
 Factor Ops : 2.854e+06 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter

# Fixed-charge problem

## Model

In [53]:
model_with_fixed_charge = gp.Model("Ewaste_with_fixed_charge", env=env)
model_with_fixed_charge.ModelSense = GRB.MAXIMIZE

## Decision variables

In [54]:
flow_shredder_to_oxide_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    shredder_to_oxide_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_shredder_to_oxide"
)

flow_oxide_to_fluoride_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    oxide_to_fluoride_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_oxide_to_fluoride"
)

flow_fluoride_to_metal_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    fluoride_to_metal_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_fluoride_to_metal"
)

shredder_loc = list(shredders.keys())
waste_to_shredder_index = pd.MultiIndex.from_product([waste_volume.index, shredder_loc, year_index], names = ["from", "to", "year"])
flow_waste_to_shredder_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    waste_to_shredder_index,
    vtype = GRB.CONTINUOUS,
    name = "flow_waste_to_shredder"
)

flow_metal_to_magnet_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    metal_to_magnet_tc,
    vtype = GRB.CONTINUOUS,
    name = "flow_metal_to_magnet"
)

y_shredder_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    fc_shredder,
    vtype = GRB.BINARY,
    name = "y_shredder"
)

y_oxide_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    fc_oxide,
    vtype = GRB.BINARY,
    name = "y_oxide"
)

auxiliary_shredder_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    fc_shredder,
    vtype = GRB.CONTINUOUS,
    name = "auxiliary_shredder"
)

auxiliary_oxide_fixed_charge = gppd.add_vars(
    model_with_fixed_charge,
    fc_oxide,
    vtype = GRB.CONTINUOUS,
    name = "auxiliary_oxide"
)

## Constraints

### Flow balance constraints

In [55]:
shredder_balance_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_shredder_to_oxide_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv1 * (flow_waste_to_shredder_fixed_charge.groupby(['to', 'year']).sum()),
    name = "shredder_balance_fixed_charge"
)

In [56]:
oxide_balance_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_oxide_to_fluoride_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv2 * (flow_shredder_to_oxide_fixed_charge.groupby(['to', 'year']).sum()),
    name = "oxide_balance_fixed_charge"
)

In [57]:
fluoride_balance_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_fluoride_to_metal_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv3 * (flow_oxide_to_fluoride_fixed_charge.groupby(['to', 'year']).sum()),
    name = "fluoride_balance_fixed_charge"
)

In [58]:
metal_balance_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_metal_to_magnet_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    conv4 * (flow_fluoride_to_metal_fixed_charge.groupby(['to', 'year']).sum()),
    name = "metal_balance_fixed_charge"
)

### Capacity constraint

In [59]:
waste_supply_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_waste_to_shredder_fixed_charge.groupby(['from', 'year']).sum() - waste_volume,
    GRB.LESS_EQUAL,
    0,
    name = "waste_supply_limit_fixed_charge"
)

In [60]:
shredder_supply_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_shredder_to_oxide_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    shredder_cap * y_shredder_fixed_charge,
    name = "shredder_supply_limit_fixed_charge"
)

In [61]:
oxide_supply_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_oxide_to_fluoride_fixed_charge.groupby(['from', 'year']).sum(),
    GRB.LESS_EQUAL,
    oxide_cap * y_oxide_fixed_charge,
    name = "oxide_supply_limit_fixed_charge"
)

In [62]:
fluoride_supply_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_fluoride_to_metal_fixed_charge.groupby(['from', 'year']).sum() - fluoride_cap,
    GRB.LESS_EQUAL,
    0,
    name = "fluoride_supply_limit_fixed_charge"
)

In [63]:
metal_supply_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_metal_to_magnet_fixed_charge.groupby(['from', 'year']).sum() - metal_cap,
    GRB.LESS_EQUAL,
    0,
    name = "metal_supply_limit_fixed_charge"
)

In [64]:
'''
shredder_inbound_limit_fixed_charge = gppd.add_constrs(
    model_with_fixed_charge,
    flow_waste_to_shredder_fixed_charge.groupby(['to', 'year']).sum(),
    GRB.LESS_EQUAL,
    (shredder_cap / conv1) * y_shredder_fixed_charge,
    name = "shredder_inbound_limit_fixed_charge"
)
'''

'\nshredder_inbound_limit_fixed_charge = gppd.add_constrs(\n    model_with_fixed_charge,\n    flow_waste_to_shredder_fixed_charge.groupby([\'to\', \'year\']).sum(),\n    GRB.LESS_EQUAL,\n    (shredder_cap / conv1) * y_shredder_fixed_charge,\n    name = "shredder_inbound_limit_fixed_charge"\n)\n'

In [65]:
# Add the max constraints for each combination of i and t
max_constraints_shredder_record = []

for i in shredders.keys():
    for j, t in enumerate(year_index):
        if j > 0:
            gc = model_with_fixed_charge.addConstr(auxiliary_shredder_fixed_charge[i, t] >= y_shredder_fixed_charge[i, t] - y_shredder_fixed_charge[i, year_index[j-1]], 
                                                   name=f"max_constraint_shredder[{i},{t}]")
        else:
            gc = model_with_fixed_charge.addConstr(auxiliary_shredder_fixed_charge[i, t] >= y_shredder_fixed_charge[i, t], name=f"max_constraint_shredder[{i},{t}]")
        max_constraints_shredder_record.append(gc)

model_with_fixed_charge.update()
index_list = pd.MultiIndex.from_product([shredder_loc, year_index], names = ['from', 'year'])
max_constraints_shredder = pd.Series(max_constraints_shredder_record, index=index_list, name="max_constraints_shredder")

In [66]:
# Add the max constraints for each combination of i and t
max_constraints_oxide_record = []

for i in oxide_producers.keys():
    for j, t in enumerate(year_index):
        if j > 0:
            gc = model_with_fixed_charge.addConstr(auxiliary_oxide_fixed_charge[i, t] >= y_oxide_fixed_charge[i, t] - y_oxide_fixed_charge[i, year_index[j-1]], 
                                                   name=f"max_constraint_oxide[{i},{t}]")
        else:
            gc = model_with_fixed_charge.addConstr(auxiliary_oxide_fixed_charge[i, t] >= y_oxide_fixed_charge[i, t], name=f"max_constraint_oxide[{i},{t}]")
        max_constraints_oxide_record.append(gc)

model_with_fixed_charge.update()
index_list = pd.MultiIndex.from_product([list(oxide_producers.keys()), year_index], names = ['from', 'year'])
max_constraints_oxide = pd.Series(max_constraints_oxide_record, index=index_list, name="max_constraints_oxide")

## Objective function

In [67]:
revenue_fixed_charge = []
for t in year_index:
    revenue_fixed_charge.append(price[t] * (flow_metal_to_magnet_fixed_charge.loc[slice(None), slice(None), t]).sum())
revenue_fixed_charge = pd.Series(revenue_fixed_charge, index=year_index, name='revenue_fixed_charge')

conversion_cost_shredder_fixed_charge = []
for t in year_index:
    conversion_cost_shredder_fixed_charge.append((oc_shredder.loc[slice(None), t] * flow_shredder_to_oxide_fixed_charge.loc[slice(None), slice(None), t]).sum())
conversion_cost_shredder_fixed_charge = pd.Series(conversion_cost_shredder_fixed_charge, index=year_index, name='conversion_cost_shredder_fixed_charge')

conversion_cost_oxide_fixed_charge = []
for t in year_index:
    conversion_cost_oxide_fixed_charge.append((oc_oxide.loc[slice(None), t] * flow_oxide_to_fluoride_fixed_charge.loc[slice(None), slice(None), t]).sum())
conversion_cost_oxide_fixed_charge = pd.Series(conversion_cost_oxide_fixed_charge, index=year_index, name='conversion_cost_oxide_fixed_charge')

conversion_cost_fluoride_fixed_charge = []
for t in year_index:
    conversion_cost_fluoride_fixed_charge.append((oc_fluoride.loc[slice(None), t] * flow_fluoride_to_metal_fixed_charge.loc[slice(None), slice(None), t]).sum())
conversion_cost_fluoride_fixed_charge = pd.Series(conversion_cost_fluoride_fixed_charge, index=year_index, name='conversion_cost_fluoride_fixed_charge')

conversion_cost_metal_fixed_charge = []
for t in year_index:
    conversion_cost_metal_fixed_charge.append((oc_ree.loc[slice(None), t] * flow_metal_to_magnet_fixed_charge.loc[slice(None), slice(None), t]).sum())
conversion_cost_metal_fixed_charge = pd.Series(conversion_cost_metal_fixed_charge, index=year_index, name='conversion_cost_metal_fixed_charge')

transportation_cost_shredder_fixed_charge = []
for t in year_index:
    transportation_cost_shredder_fixed_charge.append((shredder_to_oxide_tc.loc[slice(None), slice(None), t] * flow_shredder_to_oxide_fixed_charge.loc[slice(None), slice(None), t]).sum())
transportation_cost_shredder_fixed_charge = pd.Series(transportation_cost_shredder_fixed_charge, index=year_index, name='transportation_cost_shredder_fixed_charge')

transportation_cost_oxide_fixed_charge = []
for t in year_index:
    transportation_cost_oxide_fixed_charge.append((oxide_to_fluoride_tc.loc[slice(None), slice(None), t] * flow_oxide_to_fluoride_fixed_charge.loc[slice(None), slice(None), t]).sum())
transportation_cost_oxide_fixed_charge = pd.Series(transportation_cost_oxide_fixed_charge, index=year_index, name='transportation_cost_oxide_fixed_charge')

transportation_cost_fluoride_fixed_charge = []
for t in year_index:
    transportation_cost_fluoride_fixed_charge.append((fluoride_to_metal_tc.loc[slice(None), slice(None), t] * flow_fluoride_to_metal_fixed_charge.loc[slice(None), slice(None), t]).sum())
transportation_cost_fluoride_fixed_charge = pd.Series(transportation_cost_fluoride_fixed_charge, index=year_index, name='transportation_cost_fluoride_fixed_charge')

transportation_cost_metal_fixed_charge = []
for t in year_index:
    transportation_cost_metal_fixed_charge.append((metal_to_magnet_tc.loc[slice(None), slice(None), t] * flow_metal_to_magnet_fixed_charge.loc[slice(None), slice(None), t]).sum())
transportation_cost_metal_fixed_charge = pd.Series(transportation_cost_metal_fixed_charge, index=year_index, name='transportation_cost_metal_fixed_charge')

transportation_cost_waste_supplier_fixed_charge = []
for t in year_index:
    transportation_cost_waste_supplier_fixed_charge.append((waste_to_shredder_tc.loc[slice(None), slice(None), t] * flow_waste_to_shredder_fixed_charge.loc[slice(None), slice(None), t]).sum())
transportation_cost_waste_supplier_fixed_charge = pd.Series(transportation_cost_waste_supplier_fixed_charge, index=year_index, name='transportation_cost_waste_supplier_fixed_charge')

fixed_cost_shredder =[]
for t in year_index:
    fixed_cost_shredder.append((fc_shredder.loc[slice(None), t] * (1 - w * is_shredder_incentive_eligible) * auxiliary_shredder_fixed_charge.loc[slice(None), t]).sum())
fixed_cost_shredder = pd.Series(fixed_cost_shredder, index=year_index, name='fixed_cost_shredder')

fixed_cost_oxide =[]
for t in year_index:
    fixed_cost_oxide.append((fc_oxide.loc[slice(None), t] * (1 - w * is_oxide_producer_incentive_eligible) * auxiliary_oxide_fixed_charge.loc[slice(None), t]).sum())
fixed_cost_oxide = pd.Series(fixed_cost_oxide, index=year_index, name='fixed_cost_oxide')

In [68]:
total_revenue_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_revenue_fixed_charge += (1 / (1+MARR)**j) * revenue_fixed_charge[t]
    
total_conversion_cost_shredder_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_shredder_fixed_charge += (1 / (1+MARR)**j) * conversion_cost_shredder_fixed_charge[t]
    
total_conversion_cost_oxide_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_oxide_fixed_charge += (1 / (1+MARR)**j) * conversion_cost_oxide_fixed_charge[t]

total_conversion_cost_fluoride_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_fluoride_fixed_charge += (1 / (1+MARR)**j) * conversion_cost_fluoride_fixed_charge[t]

total_conversion_cost_metal_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_conversion_cost_metal_fixed_charge += (1 / (1+MARR)**j) * conversion_cost_metal_fixed_charge[t]

total_transportation_cost_shredder_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_shredder_fixed_charge += (1 / (1+MARR)**j) * transportation_cost_shredder_fixed_charge[t]
    
total_transportation_cost_oxide_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_oxide_fixed_charge += (1 / (1+MARR)**j) * transportation_cost_oxide_fixed_charge[t]
    
total_transportation_cost_fluoride_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_fluoride_fixed_charge += (1 / (1+MARR)**j) * transportation_cost_fluoride_fixed_charge[t]
    
total_transportation_cost_metal_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_metal_fixed_charge += (1 / (1+MARR)**j) * transportation_cost_metal_fixed_charge[t]
    
total_transportation_cost_waste_supplier_fixed_charge = 0
for j, t in enumerate(year_index, start=1):
    total_transportation_cost_waste_supplier_fixed_charge += (1 / (1+MARR)**j) * transportation_cost_waste_supplier_fixed_charge[t]
    
total_fixed_cost_shredder = 0
for j, t in enumerate(year_index, start=1):
    total_fixed_cost_shredder += (1 / (1+MARR)**j) * fixed_cost_shredder[t]

total_fixed_cost_oxide = 0
for j, t in enumerate(year_index, start=1):
    total_fixed_cost_oxide += (1 / (1+MARR)**j) * fixed_cost_oxide[t]

In [69]:
obj_exprs_fixed_charge = total_revenue_fixed_charge - total_conversion_cost_shredder_fixed_charge - total_conversion_cost_oxide_fixed_charge - total_conversion_cost_fluoride_fixed_charge - \
                        total_conversion_cost_metal_fixed_charge - total_transportation_cost_shredder_fixed_charge - total_transportation_cost_oxide_fixed_charge - \
                        total_transportation_cost_fluoride_fixed_charge - total_transportation_cost_metal_fixed_charge - total_transportation_cost_waste_supplier_fixed_charge - \
                        total_fixed_cost_oxide - total_fixed_cost_shredder
            
model_with_fixed_charge.setObjective(obj_exprs_fixed_charge, sense=GRB.MAXIMIZE)
model_with_fixed_charge.update()

## Create heuristic solution for Fixed Charge problem

In [70]:
for i, j, t in flow_shredder_to_oxide.index:
    flow_shredder_to_oxide_fixed_charge[i,j,t].Start = flow_shredder_to_oxide[i,j,t].X
    
for i, j, t in flow_oxide_to_fluoride.index:
    flow_oxide_to_fluoride_fixed_charge[i,j,t].Start = flow_oxide_to_fluoride[i,j,t].X
    
for i, j, t in flow_fluoride_to_metal.index:
    flow_fluoride_to_metal_fixed_charge[i,j,t].Start = flow_fluoride_to_metal[i,j,t].X
    
for i, j, t in flow_metal_to_magnet.index:
    flow_metal_to_magnet_fixed_charge[i,j,t].Start = flow_metal_to_magnet[i,j,t].X
    
for i, j, t in flow_waste_to_shredder.index:
    flow_waste_to_shredder_fixed_charge[i,j,t].Start = flow_waste_to_shredder[i,j,t].X

In [71]:
model_with_fixed_charge.update()

In [72]:
for i, t in y_shredder_fixed_charge.index:
    total_flow_from_i = flow_shredder_to_oxide.loc[i, slice(None), t].gppd.X.sum()
    y_shredder_fixed_charge[i,t].Start = 1 if total_flow_from_i > 0 else 0
    
    if i == 'S01': # Always open RecycleForce
        y_shredder_fixed_charge[i,t].UB = 1
        y_shredder_fixed_charge[i,t].LB = 1
        y_shredder_fixed_charge[i,t].Start = 1

In [73]:
for i, t in y_oxide_fixed_charge.index:
    total_flow_from_i = flow_oxide_to_fluoride.loc[i, slice(None), t].gppd.X.sum()
    y_oxide_fixed_charge[i,t].Start = 1 if total_flow_from_i > 0 else 0
    
    if i == 'OP65':
        y_oxide_fixed_charge[i,t].UB = 1
        y_oxide_fixed_charge[i,t].LB = 1
        y_oxide_fixed_charge[i,t].Start = 1

In [74]:
model_with_fixed_charge.update()

In [75]:
for i in shredders.keys():
    for j, t in enumerate(year_index):
        if j > 0:
            auxiliary_shredder_fixed_charge[i, t].Start = max(0, y_shredder_fixed_charge[i, t].Start - y_shredder_fixed_charge[i, year_index[j-1]].Start)
        else:
            auxiliary_shredder_fixed_charge[i, t].Start = max(0, y_shredder_fixed_charge[i, t].Start)

In [76]:
for i in oxide_producers.keys():
    for j, t in enumerate(year_index):
        if j > 0:
            auxiliary_oxide_fixed_charge[i, t].Start = max(0, y_oxide_fixed_charge[i, t].Start - y_oxide_fixed_charge[i, year_index[j-1]].Start)
        else:
            auxiliary_oxide_fixed_charge[i, t].Start = max(0, y_oxide_fixed_charge[i, t].Start)

In [77]:
model_with_fixed_charge.update()

## Solve the fixed charge problem with warm-up heuristic

In [78]:
model_with_fixed_charge.update()
model_with_fixed_charge.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 19490 rows, 165430 columns and 340716 nonzeros
Model fingerprint: 0xdb6cea38
Variable types: 164690 continuous, 740 integer (740 binary)
Coefficient statistics:
  Matrix range     [8e-02, 2e+04]
  Objective range  [5e-01, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 8e+03]

Loaded user MIP start with objective 6.44343e+07

Presolve removed 132 rows and 292 columns
Presolve time: 0.80s
Presolved: 19358 rows, 165138 columns, 339404 nonzeros
Variable types: 163770 continuous, 1368 integer (1368 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.06s

Solved with dual simplex

Root relaxation: objective 6.529801e+07, 4470 iterations

## Analyze

In [79]:
import os
from src.utils import add_sheet_to_excelbook
output_file_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "output_data.xlsx")

In [80]:
shredder_locations = y_shredder_fixed_charge.gppd.X
shredder_locations = shredder_locations.unstack(1).sort_index(axis = 1)
shredder_locations = shredder_locations[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Shredder Locations", shredder_locations)

In [81]:
oxide_producer_locations = y_oxide_fixed_charge.gppd.X
oxide_producer_locations = oxide_producer_locations.unstack().sort_index(axis = 1)
oxide_producer_locations = oxide_producer_locations[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Oxide Producer Locations", oxide_producer_locations)

In [82]:
waste_to_shredder = flow_waste_to_shredder_fixed_charge.gppd.X
waste_to_shredder = waste_to_shredder.unstack(2).sort_index(axis = 1)
waste_to_shredder = waste_to_shredder[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Waste supplier to shredder", waste_to_shredder)

In [83]:
shredder_to_oxide_producer = flow_shredder_to_oxide_fixed_charge.gppd.X
shredder_to_oxide_producer = shredder_to_oxide_producer.unstack(2).sort_index(axis = 1)
shredder_to_oxide_producer = shredder_to_oxide_producer[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Shredder to oxide producer", shredder_to_oxide_producer)

In [84]:
oxide_to_fluoride_producer = flow_oxide_to_fluoride_fixed_charge.gppd.X
oxide_to_fluoride_producer = oxide_to_fluoride_producer.unstack(2).sort_index(axis = 1)
oxide_to_fluoride_producer = oxide_to_fluoride_producer[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "oxide to fluoride producer", oxide_to_fluoride_producer)

In [85]:
fluoride_to_metal_producer = flow_fluoride_to_metal_fixed_charge.gppd.X
fluoride_to_metal_producer = fluoride_to_metal_producer.unstack(2).sort_index(axis = 1)
fluoride_to_metal_producer = fluoride_to_metal_producer[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "fluoride to metal producer", fluoride_to_metal_producer)

In [86]:
metal_to_magnet_producer = flow_metal_to_magnet_fixed_charge.gppd.X
metal_to_magnet_producer = metal_to_magnet_producer.unstack(2).sort_index(axis = 1)
metal_to_magnet_producer = metal_to_magnet_producer[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "metal to magnet producer", metal_to_magnet_producer)

In [87]:
waste_to_shredder_transport_cost = waste_to_shredder_tc * (flow_waste_to_shredder_fixed_charge.gppd.X)
shredder_tranportation_cost = {}
shredders_labels = waste_to_shredder_transport_cost.index.get_level_values(1).unique()
for i in shredders_labels:
    for t in year_index:
        shredder_tranportation_cost[(i,t)] = waste_to_shredder_transport_cost.loc[slice(None), i, t].sum()
shredder_tranportation_cost_ = pd.Series(shredder_tranportation_cost, name="Transportation cost")
shredder_tranportation_cost_ = shredder_tranportation_cost_.unstack(1)
shredder_tranportation_cost_ = shredder_tranportation_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Shredder TC record", shredder_tranportation_cost_)

In [88]:
shredder_to_oxide_operating_cost = oc_shredder * (flow_shredder_to_oxide_fixed_charge.gppd.X)
shredder_operating_cost = {}
shredders_labels = shredder_to_oxide_operating_cost.index.get_level_values(0).unique()
for i in shredders_labels:
    for t in year_index:
        shredder_operating_cost[(i,t)] = shredder_to_oxide_operating_cost.loc[i, t, slice(None)].sum()
shredder_operating_cost_ = pd.Series(shredder_operating_cost, name="Operating cost")
shredder_operating_cost_ = shredder_operating_cost_.unstack(1)
shredder_operating_cost_ = shredder_operating_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Shredder OC record", shredder_operating_cost_)

In [89]:
shredder_to_oxide_transport_cost = shredder_to_oxide_tc * (flow_shredder_to_oxide_fixed_charge.gppd.X)
oxide_tranportation_cost = {}
oxide_producers_labels = shredder_to_oxide_transport_cost.index.get_level_values(1).unique()
for i in oxide_producers_labels:
    for t in year_index:
        oxide_tranportation_cost[(i,t)] = shredder_to_oxide_transport_cost.loc[slice(None), i, t].sum()
oxide_tranportation_cost_ = pd.Series(oxide_tranportation_cost, name="Transportation cost")
oxide_tranportation_cost_ = oxide_tranportation_cost_.unstack(1)
oxide_tranportation_cost_ = oxide_tranportation_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Oxide TC record", oxide_tranportation_cost_)

In [90]:
oxide_to_fluoride_operating_cost = oc_oxide * (flow_oxide_to_fluoride_fixed_charge.gppd.X)
oxide_producer_operating_cost = {}
oxide_producers_labels = oxide_to_fluoride_operating_cost.index.get_level_values(0).unique()
for i in oxide_producers_labels:
    for t in year_index:
        oxide_producer_operating_cost[(i,t)] = oxide_to_fluoride_operating_cost.loc[i, t, slice(None)].sum()
oxide_producer_operating_cost_ = pd.Series(oxide_producer_operating_cost, name="Operating cost")
oxide_producer_operating_cost_ = oxide_producer_operating_cost_.unstack(1)
oxide_producer_operating_cost_ = oxide_producer_operating_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Oxide OC record", oxide_producer_operating_cost_)

In [91]:
oxide_to_fluoride_transport_cost = oxide_to_fluoride_tc * (flow_oxide_to_fluoride_fixed_charge.gppd.X)
fluoride_tranportation_cost = {}
fluoride_producers_labels = oxide_to_fluoride_transport_cost.index.get_level_values(1).unique()
for i in fluoride_producers_labels:
    for t in year_index:
        fluoride_tranportation_cost[(i,t)] = oxide_to_fluoride_transport_cost.loc[slice(None), i, t].sum()
fluoride_tranportation_cost_ = pd.Series(fluoride_tranportation_cost, name="Transportation cost")
fluoride_tranportation_cost_ = fluoride_tranportation_cost_.unstack(1)
fluoride_tranportation_cost_ = fluoride_tranportation_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Fluoride TC record", fluoride_tranportation_cost_)

In [92]:
fluoride_to_metal_operating_cost = oc_fluoride * (flow_fluoride_to_metal_fixed_charge.gppd.X)
fluoride_producer_operating_cost = {}
fluoride_producers_labels = fluoride_to_metal_operating_cost.index.get_level_values(0).unique()
for i in fluoride_producers_labels:
    for t in year_index:
        fluoride_producer_operating_cost[(i,t)] = fluoride_to_metal_operating_cost.loc[i, t, slice(None)].sum()
fluoride_producer_operating_cost_ = pd.Series(fluoride_producer_operating_cost, name="Operating cost")
fluoride_producer_operating_cost_ = fluoride_producer_operating_cost_.unstack(1)
fluoride_producer_operating_cost_ = fluoride_producer_operating_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "Fluoride OC record", fluoride_producer_operating_cost_)

In [93]:
fluoride_to_metal_transport_cost = fluoride_to_metal_tc * (flow_fluoride_to_metal_fixed_charge.gppd.X)
metal_tranportation_cost = {}
metal_producers_labels = fluoride_to_metal_transport_cost.index.get_level_values(1).unique()
for i in metal_producers_labels:
    for t in year_index:
        metal_tranportation_cost[(i,t)] = fluoride_to_metal_transport_cost.loc[slice(None), i, t].sum()
metal_tranportation_cost_ = pd.Series(metal_tranportation_cost, name="Transportation cost")
metal_tranportation_cost_ = metal_tranportation_cost_.unstack(1)
metal_tranportation_cost_ = metal_tranportation_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "REE TC record", metal_tranportation_cost_)

In [94]:
metal_to_magnet_operating_cost = oc_ree * (flow_metal_to_magnet_fixed_charge.gppd.X)
metal_producer_operating_cost = {}
metal_producers_labels = metal_to_magnet_operating_cost.index.get_level_values(0).unique()
for i in metal_producers_labels:
    for t in year_index:
        metal_producer_operating_cost[(i,t)] = metal_to_magnet_operating_cost.loc[i, t, slice(None)].sum()
metal_producer_operating_cost_ = pd.Series(metal_producer_operating_cost, name="Operating cost")
metal_producer_operating_cost_ = metal_producer_operating_cost_.unstack(1)
metal_producer_operating_cost_ = metal_producer_operating_cost_[year_index].reset_index()
add_sheet_to_excelbook(output_file_path, "REE OC record", metal_producer_operating_cost_)

In [95]:
total_profit = obj_exprs_fixed_charge.getValue()
total_revenue = total_revenue_fixed_charge.getValue()
total_conversion_cost_shredder = total_conversion_cost_shredder_fixed_charge.getValue()
total_conversion_cost_oxide = total_conversion_cost_oxide_fixed_charge.getValue()
total_conversion_cost_fluoride = total_conversion_cost_fluoride_fixed_charge.getValue()
total_conversion_cost_metal = total_conversion_cost_metal_fixed_charge.getValue()
total_transportation_cost_shredder = total_transportation_cost_shredder_fixed_charge.getValue()
total_transportation_cost_oxide = total_transportation_cost_oxide_fixed_charge.getValue()
total_transportation_cost_fluoride = total_transportation_cost_fluoride_fixed_charge.getValue()
total_transportation_cost_metal = total_transportation_cost_metal_fixed_charge.getValue()
total_transportation_cost_waste = total_transportation_cost_waste_supplier_fixed_charge.getValue()
total_fixed_setup_cost_oxide = total_fixed_cost_oxide.getValue()
total_fixed_setup_cost_shredder = total_fixed_cost_shredder.getValue()

output_data_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "artifacts", "output.txt")

with open(output_data_path, 'w') as file:
    # Write each variable on a new line
    file.write(f"Total Profit: {total_profit}\n")
    file.write(f"Total revenue: {total_revenue}\n")
    file.write(f"Total fixed setup cost for shredders: {total_fixed_setup_cost_shredder}\n")
    file.write(f"Total operating cost for shredders: {total_conversion_cost_shredder}\n")
    file.write(f"Total transportaion cost for shredders: {total_transportation_cost_waste}\n")
    file.write(f"Total fixed setup cost for oxide producers: {total_fixed_setup_cost_oxide}\n")
    file.write(f"Total operational cost for oxide producers: {total_conversion_cost_oxide}\n")
    file.write(f"Total transportaion cost for oxide producers: {total_transportation_cost_shredder}\n")
    file.write(f"Total operational cost for fluoride producers: {total_conversion_cost_fluoride}\n")
    file.write(f"Total transportation cost for fluoride producers: {total_transportation_cost_oxide}\n")
    file.write(f"Total operational cost for metal producers: {total_conversion_cost_metal}\n")
    file.write(f"Total transportation cost for metal producers: {total_transportation_cost_fluoride}\n")
    

In [ ]:
shredder_locations.set_index("from", inplace=True)
selected_shredders = shredder_locations.loc[~(shredder_locations==0).all(axis=1)].index
selected_shredders_coordinates= pd.DataFrame.from_dict({s:shredders[s] for s in selected_shredders}, orient='index', columns=['Latitude', 'Longitude']).reset_index(drop=False)
add_sheet_to_excelbook(output_file_path, "Shredder Coordinates", selected_shredders_coordinates)

In [ ]:
oxide_producer_locations.set_index("from", inplace=True)
selected_oxide_producers = oxide_producer_locations.loc[~(oxide_producer_locations==0).all(axis=1)].index
selected_oxide_producers_coordinates= pd.DataFrame.from_dict({s:oxide_producers[s] for s in selected_oxide_producers}, orient='index', columns=['Latitude', 'Longitude']).reset_index(drop=False)
add_sheet_to_excelbook(output_file_path, "Oxide_producers Coordinates", selected_oxide_producers_coordinates)

In [ ]:
waste_to_shredder.set_index(["from", "to"], inplace=True)
selected_waste_source = waste_to_shredder.loc[~(waste_to_shredder==0).all(axis=1)].index.get_level_values(0).unique()
selected_waste_source_coordinates = pd.DataFrame.from_dict({s:waste_source[s] for s in selected_waste_source}, orient='index', columns=['Latitude', 'Longitude']).reset_index(drop=False)
add_sheet_to_excelbook(output_file_path, "Waste Source Coordinates", selected_waste_source_coordinates)